In [0]:
!pip install python-dotenv
from dotenv import load_dotenv
_ = load_dotenv()

In [0]:
import nest_asyncio
nest_asyncio.apply()

In [0]:
!pip install -qqq llama_index

In [0]:
from llama_index.core.tools import FunctionTool

def add(x:int, y:int):
    '''Add two integers together'''
    return x + y
def mystery(x: int, y: int):
    '''Mystery function that operates on top of two numbers'''
    return(x + y)*(x+y)

add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)


In [0]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model='gpt-3.5-turbo')
response = llm.predict_and_call([add_tool, mystery_tool], "Tell me the output of mystery function of 2 and 9", verbose=True)

In [0]:
from llama_index.core import SimpleDirectoryReader
documents  = SimpleDirectoryReader(input_files=['meta-gpt-paper.pdf']).load_data()

In [0]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [0]:
print (nodes[1].get_content(metadata_mode='all'))

In [0]:
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex(nodes)
query_engine=vector_index.as_query_engine(similarity_top_k=2)

In [0]:
from llama_index.core.vector_stores import MetadataFilters
query_engine = vector_index.as_query_engine(
    simalarity_top_k=2,
    filters = MetadataFilters.from_dicts(
      [  {"key":"page_label", "value":"2"}]
    )
)

In [0]:
response = query_engine.query("What is the purpose of MetaGPT?")
print(str(response))

In [0]:
for n in response.source_nodes:
    print (n.metadata)

In [0]:
from typing import List
from llama_index.core.vector_stores import FilterCondition
def vector_query(
    query: str,
    page_numbers: List[str]
)->str:
    ''' Perform a vector search over an index
        query(str): the string the query to be embedded
        page_numbers(List[str]): Filter by set of pages. Leave BLANK if we want to look over all pages. Otherwise, filter by set of specificed pages.
    '''
    metadata_dicts = [
        {"key": "page_label", "value":p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters = MetadataFilters.from_dicts(metadata_dicts,
                                             condition=FilterCondition.OR)
    )
    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(fn=vector_query, name='vector_tool')

In [0]:
response = llm.predict_and_call([vector_query_tool],"What are the high level results of MetaGPT as described in Page 2?", verbose=True)

In [0]:
for n in response.source_nodes:
    print (n.metadata)

In [0]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async=True
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ))

In [0]:
response = llm.predict_and_call([vector_query_tool, summary_tool],"What are the high level results of MetaGPT as described in Page 2?", verbose=True)

In [0]:
response = llm.predict_and_call([vector_query_tool, summary_tool],"explain Data structures and interface definitions page 19 and 20", verbose=True)